# Create 4 csvs with log of the whole process


- read CSVs_list_log.xlsx to get what to do
- 4 sheets: One per pair serve-db (P_SB), 1 for log data, 1 for variables
- Will read all 4 sheets at once
- Use the pairs Servers-DB indicated as name of sheet
- Name of sheet also indicate logs and vars to replace

## Import the libraries and set datetime values
- pandas, os, datetime
- datetime values to write in logs and name file

In [81]:
# Required libraries
import pandas as pd
import os
import datetime as dtm
import sys

# Datetime variables
now = dtm.datetime.now()

year, month, day = now.year, now.month, now.day
lastyear, lastmonth = now.year - 1, now.month - 1

# mk lastyear and last month strings
l_year_str = str(lastyear)
l_month_str = str(lastmonth) if lastmonth > 9 else '0' + str(lastmonth)

# mk strftime string
now_fname = now.strftime('%Y%m%d%H%M')
now_inlog = now.strftime('%b%d%H%M%S')

year, month, day = now.year, now.month, now.day
hour, mins = now.hour, now.min
month_last = month- 1

print(now, type(now))
print(lastyear, type(lastyear), lastmonth, type(lastmonth))
print(f'{year} - {month} - {day} - {hour} - {mins} - {lastmonth}')
print(now_fname)

# ok, we'll see what we need later

2023-09-11 22:09:37.068990 <class 'datetime.datetime'>
2022 <class 'int'> 8 <class 'int'>
2023 - 9 - 11 - 22 - 0001-01-01 00:00:00 - 8
202309112209


## Mk the functions to use later
1. wrt_log(file, msg): to write the log w/time stamp
2. exit_prg(msg): exit program and write log before

In [82]:
# Make a funct. to write log
def wrt_log(file, msg):
    with open(file, 'a') as f:
        f.write(f"{dtm.datetime.now().strftime('%b %d %H:%M:%S')}")
        f.write(msg +'\n')

## Exit function writing log
def exit_prg():
    # txt = f'ABORT Execution: {msg}'
    # wrt_log(file, txt)
    sys.exit(0)

## First create the log before trying anything

In [83]:
# Defining log file
log = f'files/gencsvs_{now_fname}.log'

# Won't use try cause can't registers errors en log!
ln = f' > Starting {log}'
wrt_log(log, ln)

## Read whole .xlsx and get de df and P_SB

In [85]:
xlsx = 'files/CSVs_list_logM.xlsx'

try:
    dic_dfs = pd.read_excel(xlsx, sheet_name=None)
except Exception as e:
    ln = f'{dtm.datetime.now()} > ERROR:! {e}'
    wrt_log(log, ln)
    #exit_prg()
else:
    ln = f' > {xlsx} red {len(dic_dfs)} sheets: {dic_dfs.keys()}'
    wrt_log(log, ln)


## First read Fechas to get values to replace

In [93]:
fechas_df = dic_dfs['Fechas']
display(fechas_df)
f_corte = str(fechas_df.loc[0, 'Fecha de Corte'])
print(f_corte, type(f_corte))
period = f_corte[:6]
print(period, type(period))
f_corte_12a = str(int(f_corte[:4]) - 1) + f_corte[4:]
print(f_corte_12a, type(f_corte_12a))

,Fecha de Corte,Periodo,corte del mes cuyo Deterioro se calcula,corte del mismo mes del año anterior (12 meses antes)
0,20220630,202206,?,20210630


20220630 <class 'str'>
202206 <class 'str'>
20210630 <class 'str'>


## Ahora que leer Pares_S-B sheet 
- para saber todas las conexiones a bases que necesito
- y cada linea será un df 

# LLego a la conclusión que es mejor una sola tabla con todos los datos
- y luego deberé separar por par Server Base y procesar por sib dataframes

##  Use AdventureWorks DB
Till the user for MTM-BT DB is ready

In [ ]:
# Import necessary libraries
import pyodbc
import pandas as pd
import os

#pyodbc.__version__
pd.__version__
#os.__dict__
#dt.__dict__
#dir(pyodbc)

In [ ]:
### Connect to AdventureWorks2019 DB, SQL Srv authentication 

# Valid values for the connection string
driver = '{ODBC Driver 17 for SQL Server}'
server = '(local)'
dbname = 'AdventureWorks2019'
user = 'user1'
passwd = 'pass1'

# Construct the Connection String
cnx_string = f'DRIVER={driver};SERVER={server};' +\
            f'DATABASE={dbname};UID={user};PWD={passwd}'
print('Connection String:\n', cnx_string)

# Establish the connection
try:
    cnx = pyodbc.connect(cnx_string)
    cur = cnx.cursor()
    print('SUCCESS: Connection Established')
except pyodbc.Error as e:
    print('ERROR:', e)


In [ ]:
## Read excel
file = 'files/CSVs_list_simple.xlsx'

try:
    lst_df = pd.read_excel(file)
except Exception as e:
    print(f'ERROR!: {e}')
else:
    display(lst_df)


## Replace de variable values in lst_df
1. fechas in 'Modo de armado' col
2. period_value in Carpeta col

In [ ]:
## Vars to replace
f_corte = '20220630'
period = f_corte[:6]
print(f_corte, period)

In [ ]:
## 1. Replace f_corte in 'Modo de armado' col
lst_df['Modo de armado'] = lst_df['Modo de armado'].str.replace(
    'f_corte', f_corte)

lst_df.iloc[0, 1]

In [ ]:
## 2. Replace periodo in Carpeta Field
lst_df['Carpeta'] = lst_df['Carpeta'].str.replace('periodo', period)

lst_df

## Functions to make the work
- get_row_vals(row_num, df), a function that read the row an extract:
    - csvfn, query, and path (Disco :/ Carpeta /)
    - return for values as a tuple
- mk_folder(row_num, df), a funct tha create the folder where to store the cvs file
- query_to_df(row_num, df, cursor), a funct. that make the query adn save the cvs file

In [ ]:
def mk_folder(df, row_num):
    disk = df.loc[row_num, 'Disco']
    folder = df.loc[row_num, 'Carpeta']
    path = f'{disk}:/{folder}/'
    try:
        os.makedirs(path)
        print(f'Carpeta: "{path}" creada!')
    except FileExistsError:
        print(f'Carpeta: "{folder}" EXISTE!')
    return path

def qry_to_df(df, row_num):
    query = df.loc[row_num, 'Modo de armado']
    query = query.replace("’", "'")
    cur.execute(query)
    cols = [i[0] for i in cur.description]
    dats = [list(xx) for xx in cur]
    df = pd.DataFrame(data=dats, columns=cols)
    return df

filenm = f"{lst_df.loc[0, 'Nombre archivo CSV']}.csv"
filenm


## Veamos como hacemos el bucle de todo el df
1. Get total rows
2. Mk the loop in all rows of the df, that:
    - mk_folder and obtain file path
    - get the csv as a df
    - write csv (wo a funct.)

In [ ]:
rows = len(lst_df)

for row in range(rows):
    f_path = mk_folder(lst_df, row)
    csvdf = qry_to_df(lst_df, row)
    # write csv
    path_csv = f"{f_path}/{lst_df.loc[row, 'Nombre archivo CSV']}.csv"
    print(path_csv)
    try:
        csvdf.to_csv(path_csv, sep=';', index=False)
    except Exception as e:
        print(f'ERROR:!, {e}')
    else:
        print(os.listdir(f_path))
